In [1]:
pip install -r ../../requirements.txt

You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import ezkl
import torch
from torch import nn
import json
import os
import time
import scipy
import numpy as np
import matplotlib.pyplot as plt
import statistics
import math

In [4]:
%run -i ../../zkstats/core.py

In [5]:
# init path
os.makedirs(os.path.dirname('shared/'), exist_ok=True)
os.makedirs(os.path.dirname('prover/'), exist_ok=True)
verifier_model_path = os.path.join('shared/verifier.onnx')
prover_model_path = os.path.join('prover/prover.onnx')
verifier_compiled_model_path = os.path.join('shared/verifier.compiled')
prover_compiled_model_path = os.path.join('prover/prover.compiled')
pk_path = os.path.join('shared/test.pk')
vk_path = os.path.join('shared/test.vk')
proof_path = os.path.join('shared/test.pf')
settings_path = os.path.join('shared/settings.json')
srs_path = os.path.join('shared/kzg.srs')
witness_path = os.path.join('prover/witness.json')
# this is private to prover since it contains actual data
comb_data_path = os.path.join('prover/comb_data.json')

=======================  ZK-STATS FLOW =======================

In [6]:
data_path = os.path.join('data.json')
dummy_data_path = os.path.join('shared/dummy_data.json')

f_raw_input = open(data_path, "r")
data = json.loads(f_raw_input.read())["input_data"][0]
data_tensor = torch.reshape(torch.tensor(data),(1, len(data), 1))


dummy_data = np.round(np.random.uniform(1,10,len(data)),1)
json.dump({"input_data":[dummy_data.tolist()]}, open(dummy_data_path, 'w'))

dummy_data_tensor = torch.reshape(torch.tensor(dummy_data), (1, len(dummy_data),1 ))
dummy_theory_output = torch.exp(torch.mean(torch.log(dummy_data_tensor)))



In [7]:
# Verifier/ data consumer side:
class verifier_model(nn.Module):
    def __init__(self):
        super(verifier_model, self).__init__()
        # self.w = nn.Parameter(data = dummy_theory_output, requires_grad = False)

    def forward(self,X):
        return (torch.tensor(1), torch.exp(torch.mean(torch.log(X))))
        return (torch.abs((torch.log(self.w)*X.size()[1])-torch.sum(torch.log(X)))<=torch.log(torch.tensor(1.01)), self.w)

verifier_define_calculation(verifier_model, verifier_model_path, [dummy_data_path])

/var/folders/89/y9dw12v976ngdmqz4l7wbsnr0000gn/T/ipykernel_45124/2554060029.py:8: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  return (torch.tensor(1), torch.exp(torch.mean(torch.log(X))))


In [8]:
# prover calculates settings, send to verifier
# In non-linearity, scale greatly affects lookup size
theory_output = torch.exp(torch.mean(torch.log(data_tensor)))
print("Theory output: ", theory_output)
class prover_model(nn.Module):
    def __init__(self):
        super(prover_model, self).__init__()
        # w represents mean in this case
        # self.w = nn.Parameter(data = theory_output, requires_grad = False)

    def forward(self,X):
        return (torch.tensor(1), torch.exp(torch.mean(torch.log(X))))
        return (torch.abs((torch.log(self.w)*X.size()[1])-torch.sum(torch.log(X)))<=torch.log(torch.tensor(1.01)), self.w)


prover_gen_settings([data_path], comb_data_path, prover_model,prover_model_path, [8], "resources", settings_path)

Theory output:  tensor(47.6981)
==== Generate & Calibrate Setting ====
scale:  [8]
setting:  {"run_args":{"tolerance":{"val":0.0,"scale":1.0},"input_scale":8,"param_scale":8,"scale_rebase_multiplier":10,"lookup_range":[0,593654],"logrows":20,"num_inner_cols":2,"variables":[["batch_size",1]],"input_visibility":{"Hashed":{"hash_is_public":true,"outlets":[]}},"output_visibility":"Public","param_visibility":"Private"},"num_rows":14432,"total_assignments":905,"total_const_size":0,"model_instance_shapes":[[1],[1]],"model_output_scales":[0,16],"model_input_scales":[8],"module_sizes":{"kzg":[],"poseidon":[14432,[1]],"elgamal":[0,[0]]},"required_lookups":[{"Recip":{"scale":256.0}},{"Ln":{"scale":256.0}},{"Exp":{"scale":65536.0}}],"check_mode":"UNSAFE","version":"7.0.0","num_blinding_factors":null}


/var/folders/89/y9dw12v976ngdmqz4l7wbsnr0000gn/T/ipykernel_45124/2872853185.py:12: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  return (torch.tensor(1), torch.exp(torch.mean(torch.log(X))))


In [9]:
# Here verifier & prover can concurrently call setup since all params are public to get pk. 
# Here write as verifier function to emphasize that verifier must calculate its own vk to be sure
verifier_setup(verifier_model_path, verifier_compiled_model_path, settings_path,vk_path, pk_path )

print("=======================================")
# Prover generates proof
print("Theory output: ", theory_output)
prover_gen_proof(prover_model_path, comb_data_path, witness_path, prover_compiled_model_path, settings_path, proof_path, pk_path)

spawning module 0
spawning module 2


==== setting up ezkl ====


spawning module 0
spawning module 2


Time setup: 197.65462398529053 seconds
Theory output:  tensor(47.6981)
!@# compiled_model exists? True
!@# compiled_model exists? True
==== Generating Witness ====


spawning module 0


witness boolean:  1.0
witness result 1 : 92.68629455566406
==== Generating Proof ====


spawning module 2


proof:  {'instances': [[[12773529019820117503, 7223165032241168505, 6388811546753741567, 1325363978837834469], [12436184717236109307, 3962172157175319849, 7381016538464732718, 1011752739694698287], [9340507686675262708, 12576478094795452846, 16595392862476743742, 1007129139500821923]]], 'proof': '2393f5aa03eb0bae5e130928bc8bcb40a032b28c313b39a8fed41d66112ef0d20380a99a929e27ea3371b33d2fb01c72f990534e984b5a947d1464f2167a878c09a2de0046e77e19bb8927db7f7d1ad621ce470d8d7ba62eafd18602fdcacea92223ed052010e9bf42825bf321f4c7d9bf92063254dba48f96daad3f99df0346261814239c08cdbac285c236ccbdd9e686ff9fc851d23b671aaac312e2cc42912c585b9233c77f27b2d1bc642c82d4d97dddf0e627e0728249e96a7a6d37254d071e5f65196ed0e13f6e26ce10b4118e50ed4b0bca06509db33ece2db3ef97b123f69b48e5222150d3666243ade05672d7a6fc64bc5a05fcbe64a228f98ee6d32f94b2312727abf0698e2af9ad4c46c599066ff1cbfcba433fc1ceafec5f0dfb27aab2badab79f16e68ce9c28e25bea3f7289b4b61b218b56d1f5b94e583958d2065ef63770fc8b38d07bb8c549af9124eb1684c898072ad8861c03aee3860

In [11]:
# Verifier verifies
verifier_verify(proof_path, settings_path, vk_path)

num_inputs:  1
prf instances:  [[[12773529019820117503, 7223165032241168505, 6388811546753741567, 1325363978837834469], [12436184717236109307, 3962172157175319849, 7381016538464732718, 1011752739694698287], [9340507686675262708, 12576478094795452846, 16595392862476743742, 1007129139500821923]]]
proof boolean:  1.0
proof result 1 : 92.68629455566406
verified
